# Preamble

### Imports

In [45]:
import os
# os.chdir("/users/sven/Documents/code/threepoint/python_scripts/")

# from file_loader import get_kappa_millennium
# from compute_aperture_mass import aperture_mass_computer
import numpy as np
import sys
import multiprocessing as mp
from itertools import permutations
import os
# from astropy.convolution import convolve_fft
from scipy import ndimage
from scipy.signal import fftconvolve,correlate
from astropy.io import fits
import warnings

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.integrate import quad,trapz
# import treecorr
from itertools import permutations
from tqdm.notebook import tqdm,trange
import sys
from matplotlib import colors
from math import log as mlog

import shutil



### Nice plots

In [6]:
plt.rcParams['figure.figsize']        = 8., 6.
plt.rcParams['figure.dpi']            = 100
plt.rcParams['figure.subplot.left']   = 0.125
plt.rcParams['figure.subplot.right']  = 0.9
plt.rcParams['figure.subplot.bottom'] = 0.125
plt.rcParams['figure.subplot.top']    = 0.9
plt.rcParams['axes.labelsize']        = 18
plt.rcParams['axes.titlesize']        = 18
plt.rcParams['xtick.top']             = True
plt.rcParams['xtick.bottom']          = True
plt.rcParams['ytick.left']            = True
plt.rcParams['ytick.right']           = True
plt.rcParams['xtick.direction']       = 'in'
plt.rcParams['ytick.direction']       = 'in'
plt.rcParams['xtick.labelsize']       = 18
plt.rcParams['ytick.labelsize']       = 18
plt.rcParams['xtick.major.pad']       = 6.
plt.rcParams['xtick.minor.pad']       = 6.
plt.rcParams['ytick.major.pad']       = 6.
plt.rcParams['ytick.minor.pad']       = 6.
plt.rcParams['xtick.major.size']      = 6. # major tick size in points
plt.rcParams['xtick.minor.size']      = 3. # minor tick size in points
plt.rcParams['ytick.major.size']      = 6. # major tick size in points
plt.rcParams['ytick.minor.size']      = 3. # minor tick size in points
plt.rcParams['text.usetex']           = True
plt.rcParams['font.family']           = 'serif'
#plt.rcParams['font.serif']            = 'Computer Modern Roman Bold'
plt.rcParams['font.size']             = 18


### General functions

In [38]:
def progressBar(name, value, endvalue, bar_length = 25, width = 20):
    """
    Displays a progress bar with "name", tracking the progess of value/endvalue
    """
    percent = float(value) / endvalue
    arrow = '-' * int(round(percent*bar_length) - 1) + '>'
    spaces = ' ' * (bar_length - len(arrow))
    sys.stdout.write("\r{0: <{1}} : [{2}]{3}%".format(name, width, arrow + spaces, int(round(percent*100))))
    sys.stdout.flush()
    if value == endvalue:
         sys.stdout.write('\n\n')


def Efunction(theta,L=60.):
    theta = theta/L
    if(hasattr(theta,"__len__")):
        result = np.zeros_like(theta)
        result[(theta<=1)] = (np.pi+ (theta[(theta<=1)]-4)*theta[(theta<=1)])/np.pi
        result[(theta>1)] = (2*((-1) + 2*np.sqrt(1 - theta[(theta>1)]**(-2))*theta[(theta>1)] \
                                - (theta[(theta>1)]**2)/2 - np.arccos(theta[(theta>1)]**(-1)) \
                                + np.arcsin(theta[(theta>1)]**(-1))))/np.pi
        result[(theta>=np.sqrt(2))] = 0
    else:
        if(theta<=1):
            result = (np.pi+ (theta-4)*theta)/np.pi
        if(theta>1):
            result = (2*((-1) + 2*np.sqrt(1 - theta**(-2))*theta \
                                - (theta**2)/2 - np.arccos(theta**(-1)) \
                                + np.arcsin(theta**(-1))))/np.pi
        if(theta>=np.sqrt(2)):
            result = 0
    return result

def index_of(theta_ap):
    return int(round(mlog(theta_ap,2)))-1


def standfracstr(s1,s2):
    x = s1+"-"+s2
    y = r"\sqrt{"+s1+"^2+"+s2+"^2}"
    return r"$\frac{"+x+"}{"+y+"}$"

def get_theta_combinations(theta_ap,return_str=True):
    n_ap = len(theta_ap)
    aperture_radii = []
    for i in range(n_ap):
        for j in range(i,n_ap):
            for k in range(j,n_ap):
                if return_str:
                    aperture_radii.append(r"$({}',{}',{}')$".format(theta_ap[i],theta_ap[j],theta_ap[k]))
    return aperture_radii

## functions for calculating Ti

In [34]:
def map_map_cubed(th1,th2,th3,th4,th5,th6):
    idx1 = index_of(th1)
    idx2 = index_of(th2)
    idx3 = index_of(th3)
    idx4 = index_of(th4)
    idx5 = index_of(th5)
    idx6 = index_of(th6)

    return results_map_map_mean[idx1,idx2]*results_map_map_mean[idx3,idx4]*results_map_map_mean[idx5,idx6]

def map_map_cubed_T2(th1,th2,th3,th4,th5,th6):
    idx1 = index_of(th1)
    idx2 = index_of(th2)
    idx3 = index_of(th3)
    idx4 = index_of(th4)
    idx5 = index_of(th5)
    idx6 = index_of(th6)

    return map_squared_mean[idx1,idx2]*map_squared_mean[idx3,idx4]*results_map_map_mean[idx5,idx6]

def map_map_mapsq_mapsq_T5(th1,th2,th3,th4,th5,th6):
    idx1 = index_of(th1)
    idx2 = index_of(th2)
    idx3 = index_of(th3)
    idx4 = index_of(th4)
    idx5 = index_of(th5)
    idx6 = index_of(th6)

    return results_map_map_mean[idx1,idx2]*results_mapsq_mapsq_mean[idx3,idx4,idx5,idx6]

def map_map_map_mapcu_T6(th1,th2,th3,th4,th5,th6):
    idx1 = index_of(th1)
    idx2 = index_of(th2)
    idx3 = index_of(th3)
    idx4 = index_of(th4)
    idx5 = index_of(th5)
    idx6 = index_of(th6)

    return map_squared_mean[idx1,idx2]*results_map_mapcu_mean[idx3,idx4,idx5,idx6]


In [35]:
def calculateT1(th1,th2,th3,th4,th5,th6,L):
    mysum = map_map_cubed(th1,th4,th2,th5,th3,th6) \
                    +   map_map_cubed(th1,th4,th2,th6,th3,th5) \
                    +   map_map_cubed(th1,th5,th2,th4,th3,th6) \
                    +   map_map_cubed(th1,th5,th2,th6,th3,th4) \
                    +   map_map_cubed(th1,th6,th2,th4,th3,th5) \
                    +   map_map_cubed(th1,th6,th2,th5,th3,th4) 

    integrand = thetas_mean*Efunction(thetas_mean,L)*mysum
    result = trapz(integrand,thetas_mean)
    return 2*np.pi/(L*L)*result

def calculateT2(th1,th2,th3,th4,th5,th6,L):
    mysum = map_map_cubed_T2(th1,th2,th4,th5,th3,th6) \
                        + map_map_cubed_T2(th1,th2,th4,th6,th3,th5) \
                        + map_map_cubed_T2(th1,th2,th5,th6,th3,th4) \
                        + map_map_cubed_T2(th1,th3,th4,th5,th2,th6) \
                        + map_map_cubed_T2(th1,th3,th4,th6,th2,th5) \
                        + map_map_cubed_T2(th1,th3,th5,th6,th2,th4) \
                        + map_map_cubed_T2(th2,th3,th4,th5,th1,th6) \
                        + map_map_cubed_T2(th2,th3,th4,th6,th1,th5) \
                        + map_map_cubed_T2(th2,th3,th5,th6,th1,th4)
    integrand = thetas_mean*Efunction(thetas_mean,L)*mysum
    result = trapz(integrand,thetas_mean)
    return 2*np.pi/(L*L)*result

def calculateT5raw(th1,th2,th3,th4,th5,th6,L):
    mysum = map_map_mapsq_mapsq_T5(th1,th4,th2,th3,th5,th6) \
                        + map_map_mapsq_mapsq_T5(th1,th5,th2,th3,th4,th6) \
                        + map_map_mapsq_mapsq_T5(th1,th6,th2,th3,th4,th5) \
                        + map_map_mapsq_mapsq_T5(th2,th4,th1,th3,th5,th6) \
                        + map_map_mapsq_mapsq_T5(th2,th5,th1,th3,th4,th6) \
                        + map_map_mapsq_mapsq_T5(th2,th6,th1,th3,th4,th5) \
                        + map_map_mapsq_mapsq_T5(th3,th4,th1,th2,th5,th6) \
                        + map_map_mapsq_mapsq_T5(th3,th5,th1,th2,th4,th6) \
                        + map_map_mapsq_mapsq_T5(th3,th6,th1,th2,th4,th5)
    integrand = thetas_mean*Efunction(thetas_mean,L)*mysum
    result = trapz(integrand,thetas_mean)
    return 2*np.pi/(L*L)*result


def calculateT6raw(th1,th2,th3,th4,th5,th6,L):
    mysum = map_map_map_mapcu_T6(th1,th2,th3,th4,th5,th6) \
                        + map_map_map_mapcu_T6(th1,th3,th2,th4,th5,th6) \
                        + map_map_map_mapcu_T6(th2,th3,th1,th4,th5,th6) \
                        + map_map_map_mapcu_T6(th4,th5,th6,th1,th2,th3) \
                        + map_map_map_mapcu_T6(th4,th6,th5,th1,th2,th3) \
                        + map_map_map_mapcu_T6(th5,th6,th4,th1,th2,th3) 
    integrand = thetas_mean*Efunction(thetas_mean,L)*mysum
    result = trapz(integrand,thetas_mean)
    return 2*np.pi/(L*L)*result


def calculateT4(th1,th2,th3,th4,th5,th6,L):
    idx1 = index_of(th1)
    idx2 = index_of(th2)
    idx3 = index_of(th3)
    idx4 = index_of(th4)
    idx5 = index_of(th5)
    idx6 = index_of(th6)

    mysum =   results_mapsq_map_mean[idx1,idx2,idx4]*results_mapsq_map_mean[idx5,idx6,idx3] \
            + results_mapsq_map_mean[idx1,idx2,idx5]*results_mapsq_map_mean[idx4,idx6,idx3] \
            + results_mapsq_map_mean[idx1,idx2,idx6]*results_mapsq_map_mean[idx4,idx5,idx3] \
            + results_mapsq_map_mean[idx1,idx3,idx4]*results_mapsq_map_mean[idx5,idx6,idx2] \
            + results_mapsq_map_mean[idx1,idx3,idx5]*results_mapsq_map_mean[idx4,idx6,idx2] \
            + results_mapsq_map_mean[idx1,idx3,idx6]*results_mapsq_map_mean[idx4,idx5,idx2] \
            + results_mapsq_map_mean[idx4,idx5,idx1]*results_mapsq_map_mean[idx2,idx3,idx6] \
            + results_mapsq_map_mean[idx4,idx6,idx1]*results_mapsq_map_mean[idx2,idx3,idx5] \
            + results_mapsq_map_mean[idx5,idx6,idx1]*results_mapsq_map_mean[idx2,idx3,idx4]
                    
                    
    integrand = thetas_mean*Efunction(thetas_mean,L)*mysum
    result = trapz(integrand,thetas_mean)
    return 2*np.pi/(L*L)*result

def calculateT7(th1,th2,th3,th4,th5,th6,L):
    idx1 = index_of(th1)
    idx2 = index_of(th2)
    idx3 = index_of(th3)
    idx4 = index_of(th4)
    idx5 = index_of(th5)
    idx6 = index_of(th6)
    integrand = thetas_mean*Efunction(thetas_mean,L)*results_mapcu_mapcu_mean[idx1,idx2,idx3,idx4,idx5,idx6]
    result = trapz(integrand,thetas_mean)
    return 2*np.pi/(L*L)*result


## Define functions for $T_{PPP,2}$ and $T_{PT,2}$ via means of fields

In [10]:
def map_map_cubed_T2_meanfields(th1,th2,th3,th4,th5,th6):
    idx1 = index_of(th1)
    idx2 = index_of(th2)
    idx3 = index_of(th3)
    idx4 = index_of(th4)
    idx5 = index_of(th5)
    idx6 = index_of(th6)
    
    return map_squared_mean[idx1,idx2]*map_squared_mean[idx3,idx4] \
                *np.mean(map_single[:,idx5]*map_single[:,idx6],axis=0)

def map_map_map_mapcu_T6_meanfields(th1,th2,th3,th4,th5,th6):
    idx1 = index_of(th1)
    idx2 = index_of(th2)
    idx3 = index_of(th3)
    idx4 = index_of(th4)
    idx5 = index_of(th5)
    idx6 = index_of(th6)

    return map_squared_mean[idx1,idx2]*np.mean(map_single[:,idx3]*map_cubed[:,idx4,idx5,idx6],axis=0)


## Functions for calculating Gaussian covariance

In [29]:
def calculateGaussianMap2Covariance(th1,th2,L):
    idx1 = index_of(th1)
    idx2 = index_of(th2)

    mysum = results_map_map_mean[idx1,idx2]**2
    integrand = thetas_mean*Efunction(thetas_mean,L)*mysum
    result,err = trapz(integrand,thetas_mean)
#     *2 to account for permutations
    return 2*2*np.pi/(L*L)*result

def calculateGaussianMap2Covariance_test(th1,th2,L):
    idx1 = index_of(th1)
    idx2 = index_of(th2)
    result = np.mean((map_single[:,idx1]*map_single[:,idx2])**2,axis=0)
#     *2 to account for permutations
    return 2*result

def calculateNonGaussianMap2Covariance(th1,th2,L):
    idx1 = index_of(th1)
    idx2 = index_of(th2)
    idx3 = index_of(th3)
    idx4 = index_of(th4)

    mysum = results_mapsq_mapsq_mean[idx1,idx1,idx2,idx2]
    integrand = thetas_mean*Efunction(thetas_mean,L)*mysum
    result,err = trapz(integrand,thetas_mean)
    return 2*np.pi/(L*L)*result


# Read in either MS or SLICS

In [41]:
used_sim = "SLICS" #MS or SLICS

### define the correct L (if the calculated covariance is wrong check if you have done this)

In [42]:
if used_sim=="MS":
    L = (4*60-8*16)/60/180*np.pi
elif used_sim=="SLICS":
    L = (10*60-8*16)/60/180*np.pi
else:
    raise ValueError("Invalid used_sim. Allowed: MS,SLICS")

In [ ]:
if(used_sim=="MS"):
    all_los = np.arange(64)
    fpath = "/vol/euclid2/euclid2_raid2/sven/mapCorr_for_T4/"
elif used_sim=="SLICS":
    all_los = np.arange(74,800)
    fpath = "/vol/euclid2/euclid2_raid2/sven/mapCorr_SLICS_for_T4_kappa/"
else:
    raise ValueError("Invalid used_sim. Allowed: MS,SLICS")

    
n_los = len(all_los)

mask = np.zeros(n_los,dtype=bool)

    
all_theta_ap = [2,4,8,16]


n_theta_ap = len(all_theta_ap)
n_theta = 2000-1

results_map_map = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta))
results_mapsq_map = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta))
results_map_mapcu = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta))
results_mapsq_mapsq = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta))
results_mapcu_mapcu = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta))
mean_thetas = np.zeros(n_theta)

map_single = np.zeros((n_los,n_theta_ap))
map_squared = np.zeros((n_los,n_theta_ap,n_theta_ap))
map_cubed = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap))
map_four = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap))
map_six = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap))

for x,los in enumerate(tqdm(all_los,desc="Reading map powers")):
    try:
        map_single[x] = np.load(fpath+"map_single_los_{}.npy".format(los))
        map_squared[x] = np.load(fpath+"map_squared_los_{}.npy".format(los))
        map_cubed[x] = np.load(fpath+"map_cubed_los_{}.npy".format(los))
        map_four[x] = np.load(fpath+"map_four_los_{}.npy".format(los))
        map_six[x] = np.load(fpath+"map_six_los_{}.npy".format(los))
        gg = np.load(fpath+"ggcorr_mapcu_{}_{}_{}_mapcu_{}_{}_{}_los_{}.npy".format(all_theta_ap[-1],all_theta_ap[-1],all_theta_ap[-1],all_theta_ap[-1],all_theta_ap[-1],all_theta_ap[-1],los))

        mask[x] = True #check which los are available, only read in that ones
        
    except Exception as e:
        pass
    
map_single = map_single[mask]
map_squared = map_squared[mask]
map_cubed = map_cubed[mask]
map_four = map_four[mask]
map_six = map_six[mask]



all_los = all_los[mask]
n_los = len(all_los)

results_map_map = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta))
results_mapsq_map = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta))
results_map_mapcu = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta))
results_mapsq_mapsq = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta))
results_mapcu_mapcu = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta))
mean_thetas = np.zeros(n_theta)

mask2 = np.ones(n_los,dtype=bool) #verify that everything is available

for x,los in enumerate(tqdm(all_los,desc="Reading map map")):
    for i in range(n_theta_ap):
        for j in range(i,n_theta_ap):
            try:
                gg = np.load(fpath+"ggcorr_map_{}_map_{}_los_{}.npy".format(all_theta_ap[i],all_theta_ap[j],los))
            except Exception as e:
                print(e)
                mask2[x]=0
                continue
            results_map_map[x,i,j] = gg[1,:-1]
            results_map_map[x,j,i] = gg[1,:-1]
mean_thetas = gg[0,:-1]/180*np.pi

   


for x,los in enumerate(tqdm(all_los,desc="Reading mapsq map")):
    for i in range(n_theta_ap):
        for j in range(i,n_theta_ap):
            for k in range(n_theta_ap):
                try:
                    gg = np.load(fpath+"ggcorr_mapsq_{}_{}_map_{}_los_{}.npy".format(all_theta_ap[i],all_theta_ap[j],
                                                                         all_theta_ap[k],los))
                except Exception as e:
                    print(e)
                    mask2[x]=0
                    continue
                results_mapsq_map[x,i,j,k] = gg[1,:-1]
                results_mapsq_map[x,j,i,k] = gg[1,:-1]
    

for x,los in enumerate(tqdm(all_los,desc="Reading map mapcu")):
    for i in range(n_theta_ap):
        for j in range(i,n_theta_ap):
            for k in range(j,n_theta_ap):
                for ii in range(n_theta_ap):
                    try:
                        gg = np.load(fpath+"ggcorr_map_{}_mapcu_{}_{}_{}_los_{}.npy".format(all_theta_ap[ii],
                                                                                   all_theta_ap[i],
                                                                                   all_theta_ap[j],
                                                                             all_theta_ap[k],los))
                    except Exception as e:
                        print(e)
                        mask2[x]=0
                        continue
                    for idi,idj,idk in permutations([i,j,k]):
                        results_map_mapcu[x,ii,idi,idj,idk] = gg[1,:-1]

for x,los in enumerate(tqdm(all_los,desc="Reading mapsq mapsq")):
    for i in range(n_theta_ap):
        for j in range(i,n_theta_ap):
            for ii in range(n_theta_ap):
                for jj in range(ii,n_theta_ap):
                    try:
                        shutil.move(fpath+"ggcorr_mapsq_{}_{}_mapsq_{}_{}_los_{}.dat.npy".format(all_theta_ap[i],
                                                    all_theta_ap[j],
                                                    all_theta_ap[ii],
                                                    all_theta_ap[jj],
                                                    los),
                                   fpath+"ggcorr_mapsq_{}_{}_mapsq_{}_{}_los_{}.npy".format(all_theta_ap[i],
                                                    all_theta_ap[j],
                                                    all_theta_ap[ii],
                                                    all_theta_ap[jj],
                                                    los))
                    except Exception as e:
                        print("Could not move ","ggcorr_mapsq_{}_{}_mapsq_{}_{}_los_{}.dat.npy".format(all_theta_ap[i],
                                                    all_theta_ap[j],
                                                    all_theta_ap[ii],
                                                    all_theta_ap[jj],
                                                    los))
                    try:
                        gg = np.load(fpath+"ggcorr_mapsq_{}_{}_mapsq_{}_{}_los_{}.npy".format(all_theta_ap[i],
                                                    all_theta_ap[j],
                                                    all_theta_ap[ii],
                                                    all_theta_ap[jj],
                                                    los))
                    except Exception as e:
                        print(e)
                        mask2[x]=0
                        continue
                    for idi,idj in permutations([i,j]):
                        for idii,idjj in permutations([ii,jj]):
                            results_mapsq_mapsq[x,idi,idj,idii,idjj] = gg[1,:-1]


for x,los in enumerate(tqdm(all_los,desc="Reading mapcu mapcu")):
    for i in range(n_theta_ap):
        for j in range(i,n_theta_ap):
            for k in range(j,n_theta_ap):
                for ii in range(n_theta_ap):
                    for jj in range(ii,n_theta_ap):
                        for kk in range(jj,n_theta_ap):
                            try:
                                gg = np.load(fpath+"ggcorr_mapcu_{}_{}_{}_mapcu_{}_{}_{}_los_{}.npy".format(all_theta_ap[i],
                                                            all_theta_ap[j],
                                                            all_theta_ap[k],
                                                            all_theta_ap[ii],
                                                            all_theta_ap[jj],
                                                            all_theta_ap[kk],
                                                            los))
                            except Exception as e:
                                print(e)
                                mask2[x]=0
                                continue
                            for idi,idj,idk in permutations([i,j,k]):
                                for idii,idjj,idkk in permutations([ii,jj,kk]):
                                    results_mapcu_mapcu[x,idi,idj,idk,idii,idjj,idkk] = gg[1,:-1]
        



In [ ]:
map_single_mean = np.mean(map_single,axis=0)
map_squared_mean = np.mean(map_squared,axis=0)
map_cubed_mean = np.mean(map_cubed,axis=0)
map_four_mean = np.mean(map_four,axis=0)
map_six_mean = np.mean(map_six,axis=0)

if(np.sum(mask2)==len(mask2)):
    print("Length matches. computing average")
    _results_map_map_mean = np.mean(results_map_map,axis=0)
    _results_mapsq_map_mean = np.mean(results_mapsq_map,axis=0)
    _results_map_mapcu_mean = np.mean(results_map_mapcu,axis=0)
    _results_mapsq_mapsq_mean = np.mean(results_mapsq_mapsq,axis=0)
    _results_mapcu_mapcu_mean = np.mean(results_mapcu_mapcu,axis=0)

else:
    print("WARNING: Length does not match: {}/{}. Excluding los with missing elements".format(np.sum(mask2),len(mask2)))
    _results_map_map_mean = np.mean(results_map_map[mask2],axis=0)
    _results_mapsq_map_mean = np.mean(results_mapsq_map[mask2],axis=0)
    _results_map_mapcu_mean = np.mean(results_map_mapcu[mask2],axis=0)
    _results_mapsq_mapsq_mean = np.mean(results_mapsq_mapsq[mask2],axis=0)
    _results_mapcu_mapcu_mean = np.mean(results_mapcu_mapcu[mask2],axis=0)


In [ ]:
_mean_thetas = np.load(fpath+"theta_bins.npy")[:-1]

thetas_mean = np.insert(_mean_thetas,0,0)/180*np.pi

results_mapsq_map_mean = np.insert(_results_mapsq_map_mean,0,map_cubed_mean,axis=-1)
results_map_mapcu_mean = np.insert(_results_map_mapcu_mean,0,map_four_mean,axis=-1)
results_mapsq_mapsq_mean = np.insert(_results_mapsq_mapsq_mean,0,map_four_mean,axis=-1)
results_map_map_mean = np.insert(_results_map_map_mean,0,map_squared_mean,axis=-1)
results_mapcu_mapcu_mean = np.insert(_results_mapcu_mapcu_mean,0,map_six_mean,axis=-1)


full_results_mapsq_map = np.insert(results_mapsq_map,0,map_cubed_mean,axis=-1)
full_results_map_mapcu = np.insert(results_map_mapcu,0,map_four_mean,axis=-1)
full_results_mapsq_mapsq = np.insert(results_mapsq_mapsq,0,map_four_mean,axis=-1)
full_results_map_map = np.insert(results_map_map,0,map_squared_mean,axis=-1)
full_results_mapcu_mapcu = np.insert(results_mapcu_mapcu,0,map_six_mean,axis=-1)

## Read in the model covariance terms

In [ ]:
model = {}

if(used_sim=="MS"):
    model["T2"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/MS/cov_square_term2Numerical_sigma_0.00_n_1048456.68_thetaMax_1.87_gpu.dat")
    model["T6"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/MS/cov_square_term6Numerical_sigma_0.00_n_1048456.68_thetaMax_1.87_gpu.dat")
    model["T1"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/MS/cov_infinite_term1Numerical_sigma_0.00_n_1048456.68_thetaMax_1.87_gpu.dat")
    model["T5"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/MS/cov_infinite_term5Numerical_sigma_0.00_n_1048456.68_thetaMax_1.87_gpu.dat")
    model["T4"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/MS/cov_infinite_term4Numerical_sigma_0.00_n_1048456.68_thetaMax_1.87_gpu.dat")
    model["T7"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/MS/cov_infinite_term7Numerical_sigma_0.00_n_1048456.68_thetaMax_1.87_gpu.dat")

elif(used_sim=="SLICS"):
    model["T2"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/SLICS_singleZ/cov_square_term2Numerical_sigma_0.00_n_107987.71_thetaMax_7.87_gpu.dat")
    model["T6"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/SLICS_singleZ/cov_square_term6Numerical_sigma_0.00_n_107987.71_thetaMax_7.87_gpu.dat")
    model["T1"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/SLICS_singleZ/cov_infinite_term1Numerical_sigma_0.00_n_107987.71_thetaMax_7.87_gpu.dat")
    model["T5"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/SLICS_singleZ/cov_infinite_term5Numerical_sigma_0.00_n_107987.71_thetaMax_7.87_gpu.dat")
    model["T4"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/SLICS_singleZ/cov_infinite_term4Numerical_sigma_0.00_n_107987.71_thetaMax_7.87_gpu.dat")
    model["T7"] = np.loadtxt("/vol/euclid6/euclid6_ssd/sven/threepoint_with_laila/Map3_Covariances/SLICS_singleZ/cov_infinite_term7Numerical_sigma_0.00_n_107987.71_thetaMax_7.87_gpu.dat")
else:
    raise ValueError("Invalid used_sim. Allowed: MS,SLICS")

map_cubed_flat = np.zeros((map_cubed.shape[0],20))
counter = 0
for i in range(4):
    for j in range(i,4):
        for k in range(j,4):
            map_cubed_flat[:,counter] = map_cubed[:,i,j,k]
            counter += 1

direct_measured = {}         
direct_measured["cov"] = np.cov(map_cubed_flat,rowvar=False)
direct_measured["data"] = np.mean(map_cubed_flat,axis=0)


## Calculate Ti terms

In [ ]:
T1_sims = np.zeros((20,20))
T2_sims = np.zeros((20,20))
T4_sims = np.zeros((20,20))
T5raw_sims = np.zeros((20,20))
T6raw_sims = np.zeros((20,20))
allT_sims = np.zeros((20,20))

idx = -1
for i in range(n_theta_ap):
    for j in range(i,n_theta_ap):
        for k in range(j,n_theta_ap):
            idy = -1
            idx += 1
            for ii in range(n_theta_ap):
                for jj in range(ii,n_theta_ap):
                    for kk in range(jj,n_theta_ap):
                        idy += 1
                        T1_sims[idx,idy] = calculateT1(all_theta_ap[i],all_theta_ap[j],all_theta_ap[k],
                                                      all_theta_ap[ii],all_theta_ap[jj],all_theta_ap[kk],L)
                        T2_sims[idx,idy] = calculateT2(all_theta_ap[i],all_theta_ap[j],all_theta_ap[k],
                              all_theta_ap[ii],all_theta_ap[jj],all_theta_ap[kk],L)
                        T4_sims[idx,idy] = calculateT4(all_theta_ap[i],all_theta_ap[j],all_theta_ap[k],
                              all_theta_ap[ii],all_theta_ap[jj],all_theta_ap[kk],L)
                        T5raw_sims[idx,idy] = calculateT5raw(all_theta_ap[i],all_theta_ap[j],all_theta_ap[k],
                              all_theta_ap[ii],all_theta_ap[jj],all_theta_ap[kk],L)
                        T6raw_sims[idx,idy] = calculateT6raw(all_theta_ap[i],all_theta_ap[j],all_theta_ap[k],
                              all_theta_ap[ii],all_theta_ap[jj],all_theta_ap[kk],L)
                        allT_sims[idx,idy] = calculateT7(all_theta_ap[i],all_theta_ap[j],all_theta_ap[k],
                              all_theta_ap[ii],all_theta_ap[jj],all_theta_ap[kk],L)
                        progressBar("Calculating Ti",idx*20+idy,400)

                        

T5_sims = T5raw_sims - 3*T1_sims - T2_sims
T6_sims = T6raw_sims - 2*T2_sims

measured_corrf = {}
measured_corrf["T1"] = T1_sims
measured_corrf["T2"] = T2_sims
measured_corrf["T4"] = T4_sims
measured_corrf["T5"] = T5_sims
measured_corrf["T6"] = T6_sims
measured_corrf["T7"] = allT_sims - T1_sims - T2_sims - np.outer(direct_measured["data"],direct_measured["data"]) - T4_sims - T5_sims - T6_sims

## Plot covariances

In [ ]:
fig,ax = plt.subplots(3,2,figsize=(15,10),sharex=True,sharey="row")
plt.subplots_adjust(wspace=0,hspace=0)
for c,i in enumerate([1,2,4,5,6,7]):
    ax2 = c%2
    ax1 = c//2
    if i in [1,4,5,7]:
        inf = r"^\infty"
    else:
        inf = ""
    ax[ax1,ax2].plot(np.diag(model["T{}".format(i)]),color="C{}".format(c),label = r"$T_{}{}$".format(i,inf))
    ax[ax1,ax2].plot(np.diag(measured_corrf["T{}".format(i)]),color="C{}".format(c),label=r"$\hat{T}"+"_{}$".format(i),
            linestyle="--")
    ax[ax1,ax2].set_yscale('log')
    ax[ax1,ax2].legend(loc="lower left")
#     ax[ax1,ax2].set_xscale('log')

#     ax[2,1].plot([],color="C{}".format(c),label = r"$T_{}$".format(i))
#     ax[2,1].plot([],color="C{}".format(c),label=r"$\hat{T}"+"_{}$".format(i),
#             linestyle="--")
#     ax[2,1].plot((direct_measured["data"])**2,color='red',linestyle=':')
#     ax[2,1].plot(np.diag(allT_sims),color='purple',linestyle=':')


# ax[2,1].legend(loc="lower right",ncol=6,bbox_to_anchor=(1,-0.75))
for i in range(3):
    ax[i,0].set_ylabel(r"$T_i(\theta_1,\theta_2,\theta_3)$")
ax[2,0].set_xlabel(r"$(\theta_1,\theta_2,\theta_3)$")
ax[2,1].set_xlabel(r"$(\theta_1,\theta_2,\theta_3)$")
ax[2,0].set_xticks(np.arange(20))
ax[2,0].set_xticklabels(get_theta_combinations([2,4,8,16]),rotation=90)
ax[2,1].set_xticks(np.arange(20))
ax[2,1].set_xticklabels(get_theta_combinations([2,4,8,16]),rotation=90)


# plt.savefig("/users/sven/Sciebo/tex_files/thesis/phd_thesis/figures/ti_for_SLICS_singlez.png",dpi=200,
#            bbox_inches="tight")
plt.show()


In [ ]:
fig,ax = plt.subplots(1,3,sharex=True,sharey=True,figsize=(10,4))
covsample = direct_measured["cov"]
covtiest = allT_sims-np.outer(direct_measured["data"],direct_measured["data"])
covti = model["T1"]+model["T2"]+model["T4"]+model["T5"]+model["T6"]+model["T7"]
allmax = np.max([covsample,covtiest,covti])
allmin = np.max([np.min([covsample,covtiest,covti]),10**-21])
im=ax[0].imshow(covsample,norm=colors.LogNorm(vmin=allmin,vmax=allmax))
im=ax[1].imshow(covtiest,norm=colors.LogNorm(vmin=allmin,vmax=allmax))
im=ax[2].imshow(covti,norm=colors.LogNorm(vmin=allmin,vmax=allmax))
c1 = r"C_{\mathrm{sample}}"
c2 = r"\big(\sum_{i\neq 3}\hat{T}_i\big)"
c3 = r"\big(\sum_{i\neq 3} T_i\big)"
ax[0].set_title(r"${}$".format(c1))
ax[1].set_title(r"${}$".format(c2))
ax[2].set_title(r"${}$".format(c3))

for i in range(3):
    ax[i].set_yticks(np.arange(20))
    ax[i].set_yticklabels(get_theta_combinations([2,4,8,16]),fontsize=9)

    ax[i].set_xticks(np.arange(20))
    ax[i].set_xticklabels(get_theta_combinations([2,4,8,16]),fontsize=9,rotation=90)

plt.tight_layout()

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.9, 0.18, 0.03, 0.63])
fig.colorbar(im, cax=cbar_ax)

plt.savefig("/users/sven/Sciebo/tex_files/thesis/phd_thesis/figures/cov_ti_tiest_sample_slics_singlez.png",
           dpi=200)
plt.show()



fig,ax = plt.subplots(1,3,sharex=True,sharey=True,figsize=(10,4))

ratio1=(covsample-covti)/np.sqrt(covsample**2+covti**2)
ratio2=(covtiest-covti)/np.sqrt(covtiest**2+covti**2)
ratio3=(covsample-covtiest)/np.sqrt(covsample**2+covtiest**2)
allmax = np.max(np.abs([ratio1,ratio2,ratio3]))

im2 = ax[0].imshow(ratio1,cmap='seismic',vmin=-allmax,vmax=allmax)
im2 = ax[1].imshow(ratio2,cmap='seismic',vmin=-allmax,vmax=allmax)
im2 = ax[2].imshow(ratio3,cmap='seismic',vmin=-allmax,vmax=allmax)

ax[0].set_title(standfracstr(c1,c3))
ax[1].set_title(standfracstr(c2,c3))
ax[2].set_title(standfracstr(c1,c2))

for i in range(3):
    ax[i].set_yticks(np.arange(20))
    ax[i].set_yticklabels(get_theta_combinations([2,4,8,16]),fontsize=9)

    ax[i].set_xticks(np.arange(20))
    ax[i].set_xticklabels(get_theta_combinations([2,4,8,16]),fontsize=9,rotation=90)


plt.tight_layout()

fig.subplots_adjust(right=0.85)

cbar_ax2 = fig.add_axes([0.9, 0.15, 0.03, 0.62])

fig.colorbar(im2, cax=cbar_ax2)
